# Cálculo monopolo P(k) galaxias y likelihood con simulación genérica J-PAS-like #

Este código tiene 2 outputs principales: el modelo del P monopolo de galaxias evaluado en bines de z y k 'PgmonopoleValores', y el likelihood al comparar este modelo con datos simulados construidos siguiendo el mismo modelo con la cosmología fiducial, 'lnlike'.

El tiempo de cómputo de ambas variables es de 3 segundos en total.

In [1683]:
#Paquetes necesarios (seguramente sobra alguno)
import numpy as np
import math
import pandas as pd
import scipy as sp
import sympy
import scipy.integrate as integrate
import scipy.special as special
from mpmath import *
import numpy.testing as testing
import camb

# Parámetros cosmlógicos:
    - Constantes.
    - Parámetros fijos que no se varían en Cobaya.
    - Del fiducial (usados para los datos).
    - Fuera del fiducial, simulando los valores que podría estar sampleando Cobaya.

In [1684]:
#Valores de los parámetros cosmológicos en el fiducial.

#Constantes cosmológicas:
c = 2.99792458E5;   HJPAS = 1/(c/100);

#Parámetros cosmológicos que no se varían:
OmegakJPAS = 0; AsJPAS = 2.09052E-9; nsJPAS = 0.9626; 

#Parámetros cosmológicos directos:
hJPASFid = 0.674; c = 2.99792458E5; OmegabJPASh2Fid = 0.02212; OmegaCDMJPASh2Fid = 0.1206; 
OmegamJPASFid = 0.314170;

#Parámetros cosmológicos indirectos:
OmegabJPASFid = OmegabJPASh2Fid/hJPASFid**2; OmegaCDMJPASFid = OmegaCDMJPASh2Fid/hJPASFid**2;
OmegaLJPASFid = 1 - OmegamJPASFid;

In [1685]:
#Parámetros cosmológicos fuera del fiducial:
hJPAS = hJPASFid + hJPASFid/100;
OmegabJPASh2 = OmegabJPASh2Fid + OmegabJPASh2Fid/100;
OmegaCDMJPASh2 = OmegaCDMJPASh2Fid + OmegaCDMJPASh2Fid/100; 
OmegamJPAS = OmegamJPASFid + OmegamJPASFid/100;

#Parámetros cosmológicos indirectos fuera del fiducial:
OmegabJPAS = OmegabJPASh2/hJPAS**2; OmegaCDMJPAS = OmegaCDMJPASh2/hJPAS**2;
OmegaLJPAS = 1 - OmegamJPAS;

# Bineado de k y de z

In [1686]:
#Límites y pasos de los arrays. Escalas en unidades de h.
kminKArrayCompleto = 0.001;   kmaxKArrayCompleto = 2.4900;  pasoKArrayCompleto = 0.025;
zmin = 1.7;   zmax = 2.9;   pasoz = 0.2;

In [1687]:
#Bines de k, completos y reducidos
KArrayCompleto = np.exp( np.arange(math.log(kminKArrayCompleto), math.log(kmaxKArrayCompleto), pasoKArrayCompleto) )
KArray = KArrayCompleto[range(121,246)]

In [1688]:
#Bines de z:
za = np.arange(zmin, zmax+pasoz/2, pasoz)

 # Llamadas a CAMB:
     - Para fiducial
     - Para fuera de fiducial
     - Para z=0 (fuera del fiducial)
     - Para límites de bines de z por arriba y por abajo, zupper y zlower (fuera del fiducial)

In [1689]:
#Llamamos a CAMB para la cosmología fiducial

#Aquí nombramos los parámetros a meter de input a CAMB
parsFid = camb.CAMBparams()

#pars está divido en parámetros cosmológicos y parámetros de la power law, entre otras cosas.
#Llamamos ambos y damos los valores de JPAS
parsFid.set_cosmology(H0=100*hJPASFid, ombh2=OmegabJPASh2Fid, omch2=OmegaCDMJPASh2Fid, mnu=0, omk=OmegakJPAS, tau=0.06);
parsFid.InitPower.set_params(ns=nsJPAS, As=AsJPAS);
parsFid.set_matter_power(redshifts=za, kmax=KArrayCompleto[-1]);
#Aquí ya trabajamos con los resultados de CAMB 'results' y 'background':
resultsFid = camb.get_results(parsFid)
backgroundFid = camb.get_background(parsFid, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


In [1690]:
#Llamamos a otro CAMB leyendo datos fuera del fiducial
pars = camb.CAMBparams()
pars.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=OmegakJPAS, tau=0.06);
pars.InitPower.set_params(ns=nsJPAS, As=AsJPAS);
pars.set_matter_power(redshifts=za, kmax=KArrayCompleto[-1]);
results = camb.get_results(pars)
background = camb.get_background(pars, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


In [1691]:
#Importamos otro CAMB leyendo datos a z=0
parsz0 = camb.CAMBparams()
parsz0.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=OmegakJPAS, tau=0.06);
parsz0.InitPower.set_params(ns=nsJPAS, As=AsJPAS);
parsz0.set_matter_power(redshifts=[0], kmax=KArrayCompleto[-1]);
resultsz0 = camb.get_results(parsz0)
backgroundz0 = camb.get_background(parsz0, no_thermo=False)

In [1692]:
#Otro CAMB leyendo zaupper
parszupper = camb.CAMBparams();
parszupper.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=OmegakJPAS, tau=0.06);
parszupper.InitPower.set_params(ns=nsJPAS, As=AsJPAS);
parszupper.set_matter_power(redshifts=za+0.1, kmax=KArrayCompleto[-1]);
resultszupper = camb.get_results(parszupper);
backgroundzupper = camb.get_background(parszupper, no_thermo=False);

Note: redshifts have been re-sorted (earliest first)


In [1693]:
#Otro CAMB leyendo zalower
parszlower = camb.CAMBparams()
parszlower.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=OmegakJPAS, tau=0.06);
parszlower.InitPower.set_params(ns=nsJPAS, As=AsJPAS)
parszlower.set_matter_power(redshifts=za-0.1, kmax=KArrayCompleto[-1]);
resultszlower = camb.get_results(parszlower)
backgroundzlower = camb.get_background(parszlower, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


# P(k) materia lineal desde CAMB para z=0 #

In [1694]:
#P(k) materia a z=0:
kmatCAMBz0, zaCAMBz0, PmatCAMBz0 = resultsz0.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

In [1695]:
#Se interpola en k el P(k) a z = 0:
def PmatInterCAMB(k):
  return np.interp(k, kmatCAMBz0, PmatCAMBz0[0])

# Modelo de Kaiser #

Funciones E(z), f(z) y D(z):

In [1696]:
#Leemos el valor de CAMB para Omegam, quedándonos solo con bariones y cdm:
OmegamFid = np.array(resultsFid.get_Omega('baryon'))+np.array(resultsFid.get_Omega('cdm'))
Omegam = np.array(results.get_Omega('baryon'))+np.array(results.get_Omega('cdm'))

In [1697]:
Ez = np.sqrt( Omegam*(1+za)**3+(1-Omegam) );    EzFid = np.sqrt( OmegamFid*(1+za)**3+(1-OmegamFid) )

In [1698]:
H = HJPAS * Ez

In [1699]:
#Este es el sigma8 para los bines de z
sigma8Reves = np.array(results.get_sigma8())
sigma8 = sigma8Reves[::-1]

#sigma8 para z=0:
sigma8z0 = resultsz0.get_sigma8()[0]

In [1700]:
#Llamando a fsigma8 de CAMB y dividiendo entre sigma8 obtenemos f(z).
#La ordenación se hace de z más alto a z más bajo y la cambiamos
fReves = np.array(results.get_fsigma8())/sigma8Reves
f = fReves[::-1]

In [1701]:
#Esta otra forma de calcular f parece que tiene más sustento teórico:
#Gravitational growth index
gamma = 0.545
f = (Omegam*(1+za)**3*1/(Ez**2))**gamma

In [1702]:
#El growth factor D(z) no está directamente en CAMB, pero su forma es igual que la de sigmaR.
#Por tanto, usamos el sigmaR normalizado
#El valor de D(z) no coincide exactamente con el de los datos, debe venir de la discrepancia en f
DeReves = results.get_sigmaR(8)/resultsz0.get_sigmaR(8)[0]
De = DeReves[::-1]

Función A(z)

In [1703]:
#Bias
def bJPAS(z):
  return 0.53+0.289*(1+z)**2

In [1704]:
#Notar aquí que el sigma8 se pone por seguir la notación de Maroto, ya que luego cancela
A = De*bJPAS(za)*sigma8z0

Función R(z)

In [1705]:
R = De*f*sigma8z0

Modelo Kaiser

In [1706]:
def PKaiser(mu,k):
  return (A[0]+R[0]*mu**2)**2*PmatInterCAMB(k)/sigma8z0**2

# Fotometría #

In [1707]:
#Valor del error fotométrico (3.6%)
DeltazJPAS = 0.00364236313918151

Función sigmar(z)

In [1708]:
sigmar = DeltazJPAS*(1+za)/H

In [1709]:
def PKaiserFotometria(mu,k):
  return PKaiser(mu,k)*np.exp(-(k*mu*sigmar[0])**2)

# Fingers of God #

Parámetro dispersión velocidades. Se obtiene una vez para el fiducial

In [1710]:
#Hay discrepancias del 1% debido al D(z)
sigmap = (1/(6*np.pi**2)*(De/1)**2*integrate.quad(lambda k: PmatInterCAMB(k), KArrayCompleto[0], KArrayCompleto[len(KArrayCompleto)-1])[0])**0.5

/var/folders/1h/0trg724n2fg2pn5j02gypgzc0000gn/T/ipykernel_45540/4154778153.py:2: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  sigmap = (1/(6*np.pi**2)*(De/1)**2*integrate.quad(lambda k: PmatInterCAMB(k), KArrayCompleto[0], KArrayCompleto[len(KArrayCompleto)-1])[0])**0.5


Cómputo factor Fingers of God

In [1711]:
def FFog(mu,k):
    return 1/(1+(f[0]*k*mu*sigmap[0])**2)

P(k) de Kaiser con FFoG

In [1712]:
def PKaiserFotometriaFoG(mu,k):
    return FFog(mu,k)*PKaiserFotometria(mu,k) 

# Efecto AP #

In [1713]:
#Distancia comóvil desde CAMB
Xi = results.comoving_radial_distance(za)*hJPAS;   XiFid = resultsFid.comoving_radial_distance(za)*hJPASFid

In [1714]:
#Distancia angular. Multiplicamos por h para tener unidades h/Mpc:
DA = Xi/(1+za);   DAFid = XiFid/(1+za)

In [1715]:
#Factor de distorsión del AP:
FactorAP = DAFid**2*Ez/( DA**2*EzFid )

In [1716]:
#Factor Q distorsión:
def Q(mu):
    return ((Ez[0]**2*Xi[0]**2*mu**2-EzFid[0]**2*XiFid[0]**2*(mu**2-1))**0.5/(EzFid[0]*Xi[0]))

Mus y ks distorsionados

In [1717]:
def muObs(mu):
    return mu*Ez[0]/(EzFid[0]*Q(mu))

In [1718]:
def kObs(mu,k):
    return Q(mu)*k

P de galaxias: Kaiser con fotometría, FoG y AP

In [1719]:
#Vuelvo a escribir todos los factores ya que el efecto AP no se aplica por igual en todos
def Pg(mu,k):
    return FactorAP[0]*FFog(muObs(mu),kObs(mu,k))*(A[0]+R[0]*muObs(mu)**2)**2 * PmatInterCAMB(kObs(mu,k))/sigma8z0**2 *np.exp(-(k*mu*sigmar[0])**2)

# P galaxy monopole #

In [1720]:
#Aquí se realiza la integral del monopolo, con mu desde -1 a 1, y con el pre-factor 1/2.
#Se usa la regla del trapecio con 2000 pasos
def Pgmonopole(k):
    mu = np.arange(-1, 1, 1/1000)
    return 1/2 * integrate.trapz(Pg(mu, k), mu)

In [1721]:
#Evaluamos el Pgmonopole en el array de k
PgmonopoleValores = np.zeros(len(KArray))
for i in range(0, len(KArray)):
    PgmonopoleValores[i] = Pgmonopole(KArray[i])

# Covarianza #

Importación y lectura de las densidades

In [1722]:
ImportacionDensityHighZ = [i.strip().split() for i in open("/Users/guillermo/Desktop/DensityHighZ.dat").readlines()]

In [1723]:
DensityHighZ = np.zeros(len(ImportacionDensityHighZ));

In [1724]:
for i in range(0, len(ImportacionDensityHighZ)):
  DensityHighZ[i] = ImportacionDensityHighZ[i][1]

Definición del volumen (requiere distancia angular con unidades y binear en zupper y zlower)

In [1725]:
#Área del cielo
fsky = 0.2575;

In [1726]:
#Bines de z por arriba y por abajo:
zaupper = za+(za[[1]]-za[[0]])/2;    zalower = za-(za[[1]]-za[[0]])/2;

In [1727]:
#Distancia angular para los bines z upper y lower:
XiZaLower = resultszlower.comoving_radial_distance(zalower)*hJPAS
XiZaUpper = resultszupper.comoving_radial_distance(zaupper)*hJPAS

In [1728]:
#Definición de volumen:
Vol = 4*np.pi*fsky/3*(XiZaUpper**3-XiZaLower**3)

Definición del número de modos (requiere arrays en kupper y klower):

In [1729]:
#Bines de k por arriba y por abajo
KArrayUpper = np.zeros(len(KArray)); KArrayLower = np.zeros(len(KArray));

for i in range(0, len(KArray)-1):
  KArrayUpper[i] = KArray[i] + (KArray[i+1]-KArray[i])/2;   KArrayLower[i] = KArray[i] - (KArray[i+1]-KArray[i])/2;

KArrayUpper[-1] = KArrayUpper[-2];  KArrayLower[-1] = KArrayLower[-2];

In [1730]:
#Número de modos. Depende de las variables k1 y k2, que debe corresponderse a kupper y klower
def Nk(k1,k2):
    return Vol[0] * (4*np.pi/3*(k1**3-k2**3))/((2*np.pi)**3)

In [1731]:
#Evaluamos Nk para cada valor de nuestro array de k
NkEvaluado = np.zeros(len(KArray))
for i in range(0, len(KArray)):
    NkEvaluado[i] = Nk(KArrayUpper[i],KArrayLower[i])

Definición de la covarianza

In [1732]:
#Va a depender de k1 y k2. No me gusta mucho:
def Cov(k,k1,k2):
    return 2 * (Pgmonopole(k) + 1/DensityHighZ[0])**2 / Nk(k1,k2)

In [1733]:
#Evaluamos Cov para nuestros k
CovEvaluado = 2 *(PgmonopoleValores + 1/DensityHighZ[0])**2 / NkEvaluado

# Lectura de datos #

Importación de los de datos del P(k) monopolo que hemos generado. Para todos los bines de z

In [1734]:
ImportacionDatosHighZ = [i.strip().split() for i in open("/Users/guillermo/Desktop/FicticioHighZArrayEnK.dat").readlines()]

In [1735]:
DataPkzTotal = np.zeros(len(za));

In [1736]:
Pk1 = np.zeros(len(ImportacionDatosHighZ)); Pk2 = np.zeros(len(ImportacionDatosHighZ));
Pk3 = np.zeros(len(ImportacionDatosHighZ)); Pk4 = np.zeros(len(ImportacionDatosHighZ));
Pk5 = np.zeros(len(ImportacionDatosHighZ)); Pk6 = np.zeros(len(ImportacionDatosHighZ));
Pk7 = np.zeros(len(ImportacionDatosHighZ));

In [1737]:
#Asociamos a cada Pk la columna correspondiente en el archivo de los datos. z1 es la columna 3, es decir [2]:
for i in range(0, len(ImportacionDatosHighZ)):
  Pk1[i] = ImportacionDatosHighZ[i][2]; Pk2[i] = ImportacionDatosHighZ[i][3]; Pk3[i] = ImportacionDatosHighZ[i][4];
  Pk4[i] = ImportacionDatosHighZ[i][5]; Pk5[i] = ImportacionDatosHighZ[i][6]; Pk6[i] = ImportacionDatosHighZ[i][7];
  Pk7[i] = ImportacionDatosHighZ[i][8]

In [1738]:
#Esto vendrá bien cuando queramos usar todos los bines de z
DataPkzTotal = np.array([Pk1,Pk2,Pk3,Pk4,Pk5,Pk6,Pk7])

# Likelihood #

Bineamos el P(k) para cuando entre al likelihood:

In [1739]:
PgBineadoz1 = np.zeros(len(KArray)); PgBineadoz2 = np.zeros(len(KArray));
PgBineadoz3 = np.zeros(len(KArray)); PgBineadoz4 = np.zeros(len(KArray));
PgBineadoz5 = np.zeros(len(KArray)); PgBineadoz6 = np.zeros(len(KArray));
PgBineadoz7 = np.zeros(len(KArray));

In [1740]:
PgBineado = np.array([PgBineadoz1,PgBineadoz2,PgBineadoz3,PgBineadoz4,PgBineadoz5,PgBineadoz6,PgBineadoz7])

In [1741]:
for i in range(0, len(KArray)):
    PgBineado[0][i] = PgmonopoleValores[i]

Bineamos la covarianza para cuando entre al likelihood

In [1742]:
CovBineadoz1 = np.zeros(len(KArray)); CovBineadoz2 = np.zeros(len(KArray));
CovBineadoz3 = np.zeros(len(KArray)); CovBineadoz4 = np.zeros(len(KArray));
CovBineadoz5 = np.zeros(len(KArray)); CovBineadoz6 = np.zeros(len(KArray));
CovBineadoz7 = np.zeros(len(KArray));

In [1743]:
CovBineado = np.array([CovBineadoz1,CovBineadoz2,CovBineadoz3,CovBineadoz4,CovBineadoz5,CovBineadoz6,CovBineadoz7])

In [1744]:
for i in range(0, len(KArray)):
    CovBineado[0][i] = CovEvaluado[i]

Construimos el likelihood, siendo j el valor del array de z, i el valor del array de k:

In [1745]:
#Este likelihood es similar a un chi^2. ¿Igual? ¿Factor del log del determinante = log cov?
def lnlikeSinSumar(j,i):
    return (PgBineado[j][i]-DataPkzTotal[j][i])**2 * 1/CovBineado[j][i] + np.log(CovBineado[j][i])

índices en los que queremos evaluar el likelihood y el paso entre ellos. Usamos todos:

In [1746]:
IndicesLikelihood = np.arange(0,len(KArray),1)

In [1747]:
#Likelihood sumando a los índices
lnlike = np.sum(lnlikeSinSumar(0,IndicesLikelihood))
lnlike

1749.692599596198